<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/dynamic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
import scipy.signal as sig
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify


In [22]:
N=3
Nfeed = 2
R = 2. #reflux ratio
m0tray = 0.2 #tray holdup in moles at zero flow
m0cond = 0.5 #condenser holdup at no flow
m0boil = 0.5 #boiler holdup at no flow
m0 = np.r_[m0cond, (m0tray,)*N, m0boil]
m = np.full(N+2,0.8)  #fill up condenser, reboiler, and trays with 0.8 moles of liquid

In [16]:
m[2]=0.1

In [17]:
weir_const = 0.1 #lumped Weir constant that includes effect of weir width, density, viscoisty, conversion from volumetric to molar flows

In [ ]:
def rhs(t, vec):
    m = vec
    mdiff = m-m0
    L=weir_const*(jnp.where(mdiff<0, 0., mdiff))**(1.5)
    D = L[0]/(R+1)
    L[0] = R*D
    netflows = L        #Out is positive